In [ ]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib as mpl 
# import missingno as msno
import seaborn as sns

from src import config

In [ ]:
df = pd.read_csv(config.RAW_FILE_PATH / 'olist_order_items_dataset.csv')

print(f'''
{df.info()}
{print(df.isna().sum())}
{df.shape}
{df.head()}
''')

In [ ]:
df = df.groupby('order_id').agg({'product_id': lambda x: list(x)})

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
import pyspark.sql.types as T 
import pyspark.sql.functions as F 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

import pyspark.sql.functions as F
from pyspark.ml.fpm import FPGrowth

import findspark
findspark.init()
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# conf = SparkConf().setAppName("base").setMaster("local[*]")
# spark = SparkContext(conf=conf).getOrCreate()
# spark.stop()

In [ ]:
df = spark.read.options(header='true').csv('../data/raw/olist_order_items_dataset.csv')

df.show(2)

In [ ]:
df = df.dropDuplicates(['order_id', 'product_id']).sort('order_id')

df.show(2)

In [ ]:
df = df.groupBy('order_id').agg(F.collect_list('product_id')).sort('order_id')

df.show(2)

In [ ]:
fpGrowth = FPGrowth(itemsCol='collect_list(product_id)', minSupport=0.00001, minConfidence=0.0001)

model = fpGrowth.fit(df)

In [ ]:
model.freqItemsets.show()
items = model.freqItemsets.toPandas()

model.associationRules.show()
rules = model.associationRules.toPandas()

In [ ]:
items.to_csv('./mba_freq_items.csv')
rules.to_csv('./mba_pattern.csv')